# Reshaping your Data Frame

For this tutorial, you will need the `liked_posts.html` file from Instagram. This file is found in the `likes` directory. If you don't have this export, I've provided some fake data in `./csv/fake_data/liked_posts.csv`. 

If you need to convert your `html` file to `csv`, follow the instructions in `likes_per_day.ipynb`

In [1]:
import pandas as pd
filename = './csv/fake_data/liked_posts.csv'
df = pd.read_csv(filename, parse_dates=['date'])
uniques = df.drop_duplicates(keep='first')

uniques

,user,date
0,zeevzaeqevmeza,2014-03-21
1,oeefezepvezfeez,2014-03-21
2,zeeeefzeezeveefeqz,2014-03-21
3,efveeff,2014-03-20
4,zeeeefzeezeveefeqz,2014-03-20
...,...,...
30415,eepezopfoz,2020-01-13
30416,veepaqaeppez,2020-01-13
30417,qeppeeeqeeaeqfqzpe,2020-01-13
30418,qefozvopozoeeq,2020-01-13


### Group, Count, and Unstack

Currently, the data is just a list of accounts and dates. It may be helpful to make each row represent a single day. I'll detail 2 ways to accomplish this:
1. Create columns that represent each user and assign a 0 or 1 to represent if a post of theirs was liked on a given day
2. Create columns to store usernames whose posts were liked on a given day

For the first case, we can group the data by both `date` and `user`, then `count` and `unstack` the data to transform as desired. Since we are using a data frame with only unique rows, the maximum count for a user on a given day is 1, thus we have a 0/1 indicator of liking a users post on a date.

In [2]:
_grouped = uniques.groupby(['date', 'user'])['user']
_counts = _grouped.count()
unstacked = _counts.unstack().reset_index().rename_axis(None, axis=1).fillna(0)

unstacked

,date,007poevfmeqqfeeveeef6176,03mozqe3,1010zeqe3405,11g08em,303apeqf,30eqepameeeqqoz,36zezfoaqzve,3fqeezzfevveeep,3z4aeve,...,zvoafe,zvqeeeepp627,zzefqov531,zzeopzmveef2131,zzeveqeeoqf,zzmef,zzmezqoopzeqef,zzpezzqzeee,zzq1913,zzvoeeeaoaaeeeep
0,2012-07-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2012-07-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2012-09-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2013-07-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013-07-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261,2022-02-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2262,2022-02-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2263,2022-02-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2264,2022-02-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


For the second case, we must first add a new column to the data frame. We're going to name this column `key` and assign the value as the cumulative count of the item when grouped by `date`.

This will make the data set look something like this:
```
date | user | key
1    | A    | 1
1    | B    | 2
1    | C    | 3
1    | D    | 4
2    | E    | 1
2    | B    | 2
3    | A    | 1
3    | C    | 2
3    | D    | 3
4    | B    | 1
4    | E    | 2
```

In [3]:
_keyed = uniques.assign(key=uniques.groupby('date').cumcount() + 1)

_keyed

,user,date,key
0,zeevzaeqevmeza,2014-03-21,1
1,oeefezepvezfeez,2014-03-21,2
2,zeeeefzeezeveefeqz,2014-03-21,3
3,efveeff,2014-03-20,1
4,zeeeefzeezeveefeqz,2014-03-20,2
...,...,...,...
30415,eepezopfoz,2020-01-13,5
30416,veepaqaeppez,2020-01-13,6
30417,qeppeeeqeeaeqfqzpe,2020-01-13,7
30418,qefozvopozoeeq,2020-01-13,8


Now, we can transform the data frame using the `crosstab` method. We'll reindex the data frame using the `date` column and create columns using the `key` column. These columns will be filled with values from the `user` column.

In [4]:
_reshaped = pd.crosstab(index=_keyed.date, columns=_keyed.key, values=_keyed.user, aggfunc='first')
reshaped = _reshaped.fillna('').add_prefix('user_').reset_index().rename_axis(None, axis=1)

reshaped

,date,user_1,user_2,user_3,user_4,user_5,user_6,user_7,user_8,user_9,...,user_96,user_97,user_98,user_99,user_100,user_101,user_102,user_103,user_104,user_105
0,2012-07-03,aofqeeqezqezf,qeeqqez82,fqeezeevfveeqzeef,veefefeefqe,fqeezeevf,evvpemezezeeez,qeffeeeopqqoa,mqeeqqoeze91,veevofeoaezvz,...,,,,,,,,,,
1,2012-07-14,qeeqqez82,,,,,,,,,...,,,,,,,,,,
2,2012-09-26,fqeezeevf,,,,,,,,,...,,,,,,,,,,
3,2013-07-03,fqeezeevf,,,,,,,,,...,,,,,,,,,,
4,2013-07-04,fqeezeevf,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261,2022-02-04,qeppmzeev,,,,,,,,,...,,,,,,,,,,
2262,2022-02-05,egmezzpee,,,,,,,,,...,,,,,,,,,,
2263,2022-02-07,qeppmzeev,,,,,,,,,...,,,,,,,,,,
2264,2022-02-08,zoeqfqeefqoz,qeppmzeev,qegfqeaeqee,,,,,,,...,,,,,,,,,,


### Merge the columns to a list of strings

First, we transform the user columns to a list using the `values.tolist` method. To select these columns, we'll use the `loc` method and select all rows and columns that are not `date`.

In [5]:
user_lists = reshaped.loc[:, reshaped.columns != 'date'].values.tolist()

# user_lists

This 2D array has a lot of empty strings. Let's remove those by iterating over the elements.

In [6]:
for item in user_lists:
  while '' in item:
    item.remove('')

# user_lists

Now, we can grab the `dates` column from the data frame and merge them with the lists of users.

In [7]:
dates = reshaped.date
merged = pd.DataFrame(dates).assign(users=user_lists)

merged

,date,users
0,2012-07-03,"[aofqeeqezqezf, qeeqqez82, fqeezeevfveeqzeef, ..."
1,2012-07-14,[qeeqqez82]
2,2012-09-26,[fqeezeevf]
3,2013-07-03,[fqeezeevf]
4,2013-07-04,[fqeezeevf]
...,...,...
2261,2022-02-04,[qeppmzeev]
2262,2022-02-05,[egmezzpee]
2263,2022-02-07,[qeppmzeev]
2264,2022-02-08,"[zoeqfqeefqoz, qeppmzeev, qegfqeaeqee]"
